In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df=pd.read_csv('zomato.csv')

In [5]:
df.head(2)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari


In [7]:
df['location'].sample(5)

13851    Electronic City
3102        Banashankari
5223              Ulsoor
33847         Nagarbhavi
45400             Domlur
Name: location, dtype: object

In [8]:
df.dropna(inplace=True)

In [9]:
len(df['location'].unique())

88

In [10]:
locations=pd.DataFrame({"Name":df['location'].unique()})

In [15]:
locations['new_Name']='Bangalore '+locations['Name']

In [17]:
locations.head()

,Name,new_Name
0,Banashankari,Bangalore Banashankari
1,Basavanagudi,Bangalore Basavanagudi
2,Jayanagar,Bangalore Jayanagar
3,Kumaraswamy Layout,Bangalore Kumaraswamy Layout
4,Rajarajeshwari Nagar,Bangalore Rajarajeshwari Nagar


In [18]:
!pip install geopy

In [20]:
from geopy.geocoders import Nominatim

In [21]:
lat_lon=[]
geolocator=Nominatim(user_agent="app")
for location in locations['Name']:
    location = geolocator.geocode(location)
    if location is None:
        lat_lon.append(np.nan)
    else:    
        geo=(location.latitude,location.longitude)
        lat_lon.append(geo)

In [22]:
locations['geo_loc']=lat_lon

In [23]:
locations.head()

,Name,new_Name,geo_loc
0,Banashankari,Bangalore Banashankari,"(15.8876779, 75.7046777)"
1,Basavanagudi,Bangalore Basavanagudi,"(12.9417261, 77.5755021)"
2,Jayanagar,Bangalore Jayanagar,"(27.64392675, 83.05280519687284)"
3,Kumaraswamy Layout,Bangalore Kumaraswamy Layout,"(12.9081487, 77.5553179)"
4,Rajarajeshwari Nagar,Bangalore Rajarajeshwari Nagar,"(12.9274413, 77.5155224)"


In [24]:
locations.to_csv('zomato_locations.csv',index=False)

### We have found out latitude and longitude of each location listed in the dataset using geopy.`

In [25]:
Rest_locations=pd.DataFrame(df['location'].value_counts().reset_index())

In [26]:
Rest_locations.columns=['Name','count']
Rest_locations.head()

,Name,count
0,Koramangala 5th Block,1783
1,BTM,1456
2,Indiranagar,1350
3,HSR,1162
4,Jayanagar,1037


In [27]:
locations.head()

,Name,new_Name,geo_loc
0,Banashankari,Bangalore Banashankari,"(15.8876779, 75.7046777)"
1,Basavanagudi,Bangalore Basavanagudi,"(12.9417261, 77.5755021)"
2,Jayanagar,Bangalore Jayanagar,"(27.64392675, 83.05280519687284)"
3,Kumaraswamy Layout,Bangalore Kumaraswamy Layout,"(12.9081487, 77.5553179)"
4,Rajarajeshwari Nagar,Bangalore Rajarajeshwari Nagar,"(12.9274413, 77.5155224)"


In [28]:
locations.shape

(88, 3)

In [29]:
Rest_locations.shape

(88, 2)

In [30]:
Restaurant_locations=Rest_locations.merge(locations,on='Name',how="left").dropna()
Restaurant_locations.head()

,Name,count,new_Name,geo_loc
0,Koramangala 5th Block,1783,Bangalore Koramangala 5th Block,"(12.9343774, 77.628415)"
1,BTM,1456,Bangalore BTM,"(45.95485055, -112.49659530324134)"
2,Indiranagar,1350,Bangalore Indiranagar,"(12.9732913, 77.6404672)"
3,HSR,1162,Bangalore HSR,"(18.1475, 41.538889)"
4,Jayanagar,1037,Bangalore Jayanagar,"(27.64392675, 83.05280519687284)"


In [31]:
Restaurant_locations.shape

(87, 4)

In [32]:
Restaurant_locations['count'].max()

1783

In [33]:
type(Restaurant_locations['geo_loc'][0])

tuple

In [34]:
def generateBaseMap(default_location=[12.97, 77.59], default_zoom_start=12):
    base_map = folium.Map(location=default_location, zoom_start=default_zoom_start)
    return base_map

In [35]:
len(Restaurant_locations['geo_loc'])

87

In [36]:
Restaurant_locations.isna().sum()

Name        0
count       0
new_Name    0
geo_loc     0
dtype: int64

In [37]:
Restaurant_locations['geo_loc'][0][0]

12.9343774

In [38]:
Restaurant_locations['geo_loc'][0][1]

77.628415

In [39]:
np.array(Restaurant_locations['geo_loc'])

array([(12.9343774, 77.628415), (45.95485055, -112.49659530324134),
       (12.9732913, 77.6404672), (18.1475, 41.538889),
       (27.64392675, 83.05280519687284), (12.2655944, 76.6465404),
       (44.3730577, -71.6118577), (12.9302645, 77.6332585),
       (12.9400321, 77.6203272), (12.9552572, 77.6984163),
       (12.93433385, 77.63040639553275), (29.4534927, -98.6440138),
       (44.621621, -84.790422), (40.7153533, -74.0076212),
       (12.887979, 77.5970812), (12.9778793, 77.6246697),
       (13.0221416, 77.6403368), (14.5395813, 121.070371),
       (12.93577245, 77.66676103753434), (12.9242381, 77.6289059),
       (40.7652844, -76.373824), (24.593839, 73.6968527),
       (15.8876779, 75.7046777), (13.0027353, 77.5703253),
       (40.5763076, -74.1270894), (31.89376, -88.066644),
       (-6.2659285, 106.7842561), (33.5935063, -79.0345627),
       (51.5230784, -0.7374421), (13.0346384, 77.5681733),
       (12.996845, 77.6130165), (12.9417261, 77.5755021),
       (13.0093455, 77.6377

In [40]:
#### unzip it
lat,lon=zip(*np.array(Restaurant_locations['geo_loc']))

In [41]:
type(lat)

tuple

In [42]:
Restaurant_locations['lat']=lat
Restaurant_locations['lon']=lon

In [43]:
Restaurant_locations.head()

,Name,count,new_Name,geo_loc,lat,lon
0,Koramangala 5th Block,1783,Bangalore Koramangala 5th Block,"(12.9343774, 77.628415)",12.934377,77.628415
1,BTM,1456,Bangalore BTM,"(45.95485055, -112.49659530324134)",45.954851,-112.496595
2,Indiranagar,1350,Bangalore Indiranagar,"(12.9732913, 77.6404672)",12.973291,77.640467
3,HSR,1162,Bangalore HSR,"(18.1475, 41.538889)",18.147500,41.538889
4,Jayanagar,1037,Bangalore Jayanagar,"(27.64392675, 83.05280519687284)",27.643927,83.052805


In [ ]:
!pip install folium

In [44]:
import folium
from folium.plugins import HeatMap
basemap=generateBaseMap()

In [45]:
basemap

In [46]:
Restaurant_locations[['lat','lon','count']].values.tolist()

[[12.9343774, 77.628415, 1783.0],
 [45.95485055, -112.49659530324134, 1456.0],
 [12.9732913, 77.6404672, 1350.0],
 [18.1475, 41.538889, 1162.0],
 [27.64392675, 83.05280519687284, 1037.0],
 [12.2655944, 76.6465404, 1016.0],
 [44.3730577, -71.6118577, 824.0],
 [12.9302645, 77.6332585, 725.0],
 [12.9400321, 77.6203272, 720.0],
 [12.9552572, 77.6984163, 683.0],
 [12.93433385, 77.63040639553275, 644.0],
 [29.4534927, -98.6440138, 596.0],
 [44.621621, -84.790422, 570.0],
 [40.7153533, -74.0076212, 508.0],
 [12.887979, 77.5970812, 496.0],
 [12.9778793, 77.6246697, 477.0],
 [13.0221416, 77.6403368, 467.0],
 [14.5395813, 121.070371, 466.0],
 [12.93577245, 77.66676103753434, 462.0],
 [12.9242381, 77.6289059, 456.0],
 [40.7652844, -76.373824, 445.0],
 [24.593839, 73.6968527, 442.0],
 [15.8876779, 75.7046777, 387.0],
 [13.0027353, 77.5703253, 366.0],
 [40.5763076, -74.1270894, 351.0],
 [31.89376, -88.066644, 333.0],
 [-6.2659285, 106.7842561, 327.0],
 [33.5935063, -79.0345627, 307.0],
 [51.5230784

In [47]:
HeatMap(Restaurant_locations[['lat','lon','count']].values.tolist(),zoom=20,radius=15).add_to(basemap)

In [48]:
basemap

##### It is clear that restaurants tend to concentrate in central bangalore area.
##### The clutter of restaurants lowers are we move away from central.
##### So,potential restaurant entrepreneurs can refer this and find out good locations for their venture

#### note heatmap is good when we have latitude,longitude or imporatnce of that particular place or count of that place